In [1]:
# 분별 날짜 파일에 있는 날짜 기준으로 DB에서 데이터 받아와서 저장하기

import MySQLdb
import os
import numpy as np
import csv
import datetime

# MySQL DB 연결
db = MySQLdb.connect('210.102.142.13',"root", "witlab8*", "cas_db")
c = db.cursor()

# 분별 날짜
# date = np.genfromtxt('../data/date_sunrise_sunset.csv', delimiter=',', dtype='str')
date = np.genfromtxt('../data/test_date_sunrise_sunset.csv', delimiter=',', dtype='str')

# csv 파일로 내보내기
# w = open('../data/db_connect_data_rnn.csv', 'w', encoding='utf-8')
w = open('../data/db_connect_test_data_rnn.csv', 'w', encoding='utf-8')
wr = csv.writer(w)

data_length = []

# 각 날짜별 데이터들 DB에서 가져오고 csv 파일로 저장
# 데이터는 cct, swr, uvb, uvi 순
for j in range(len(date)):
    sql = "select time(date),cas_swr, cas_mwr, cas_lwr from natural_tracker left outer join cas_wave_ratio using(date) where date(date) = '"+ str(date[j][0]) + "' order by time(date)"
    c.execute(sql)
    rows = c.fetchall()
    
    # 일출 후 6시간과 일몰 후 6시간 데이터 사용, 일출 혹은 일몰 당시 데이터가 없을 경우 가장 가까운 다른 데이터로 변환
    sunrise = datetime.datetime.strptime(date[j][1], '%H:%M:%S')
    sunset = datetime.datetime.strptime(date[j][2], '%H:%M:%S')
    
    standard = datetime.datetime.strptime('1900-01-01 00:00:00', '%Y-%m-%d %H:%M:%S')
    
    sunrise = sunrise - standard
    sunset = sunset - standard
    
    start = 0
    end = 0

    for i in range(len(rows)):
        if(rows[i][0] >= sunrise):
            start = i
            print(date[j][0], rows[start][0], sunrise)
            break;
    
    for i in range(len(rows)):
        if(rows[i][0] >= sunset):
            end = i
            print(date[j][0], rows[end][0], sunset)
            break;
    
    last = int((end - start) / 4)
    last = last * 4
    
#     # start에 저장된 index부터 772개 데이터 가져와서 저장
    for l in range(start, start + last):
        wr.writerow([rows[l][1], rows[l][2], rows[l][3]])
    print(date[j][0] + " " + str(last) + " data saved.")
    for l in range(4):
        data_length.append(int(last/4))
    print('----------------------------------------------')
        
w.close()
db.close()

2018-04-19 5:52:00 5:52:00
2018-04-19 19:52:00 19:52:00
2018-04-19 832 data saved.
----------------------------------------------
2018-04-21 5:49:00 5:49:00
2018-04-21 19:49:00 19:49:00
2018-04-21 832 data saved.
----------------------------------------------
2018-04-26 5:43:00 5:43:00
2018-04-26 19:43:00 19:43:00
2018-04-26 832 data saved.
----------------------------------------------
2018-04-27 5:42:00 5:42:00
2018-04-27 19:42:00 19:42:00
2018-04-27 832 data saved.
----------------------------------------------
2018-04-28 5:41:00 5:41:00
2018-04-28 19:41:00 19:41:00
2018-04-28 832 data saved.
----------------------------------------------
2018-04-29 5:40:00 5:40:00
2018-04-29 19:40:00 19:40:00
2018-04-29 832 data saved.
----------------------------------------------
2018-04-30 5:38:00 5:38:00
2018-04-30 19:38:00 19:38:00
2018-04-30 832 data saved.
----------------------------------------------
2018-05-04 5:34:00 5:34:00
2018-05-04 19:34:00 19:34:00
2018-05-04 832 data saved.
-------

In [2]:
# date = np.genfromtxt('../data/date_sunrise_sunset.csv', delimiter=',', dtype='str')
date = np.genfromtxt('../data/test_date_sunrise_sunset.csv', delimiter=',', dtype='str')

# 데이터 받아오고 편차 계산히기
# import_data = np.loadtxt('../data/db_connect_data_rnn.csv', delimiter=',')
import_data = np.loadtxt('../data/db_connect_test_data_rnn.csv', delimiter=',')

one = import_data[:,0] # swr
two = import_data[:,1] # mwr
thr = import_data[:,2] # lwr

swr = []
mwr = []
lwr = []

data_index = 0

for i in range(len(data_length)):
    temp = []
    temp2 = []
    temp3 = []
    for j in range(data_length[i]):
        temp.append(one[j + data_index])
        temp2.append(two[j + data_index])
        temp3.append(thr[j + data_index])
    swr.append(temp)
    mwr.append(temp2)
    lwr.append(temp3)
    data_index += data_length[i]
    
# w = open('../data/swr_ratio_classification_rnn.csv', 'w', encoding='utf-8')
w = open('../data/swr_ratio_test_classification_rnn.csv', 'w', encoding='utf-8')
wr = csv.writer(w)

for i in range(len(data_length)):
    all_hap = 0
    swr_hap = 0
    ratio = 0
    grade = 0
    
    for j in range(len(swr[i])):
        swr_hap += swr[i][j]
        all_hap += swr[i][j] + mwr[i][j] + lwr[i][j]
        
    ratio = round(swr_hap / all_hap * 100, 2)
    
    if(ratio >= 18.00):
        grade += 1
    if(ratio >= 20.00):
        grade += 1
    if(ratio >= 22.00):
        grade += 1
    if(ratio >= 24.00):
        grade += 1
    
    wr.writerow([ratio, grade])
w.close()

In [1]:
import tensorflow as tf
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import OneHotEncoder
import matplotlib.pyplot as plt
ohe = OneHotEncoder()
tf.set_random_seed(777)

import_data = np.genfromtxt('../data/swr_ratio_classification_rnn.csv', delimiter=',', dtype='float')

x_data = []

for i in range(len(import_data)):
    temp = []
    temp.append(import_data[i][0])
    x_data.append(temp)

y_data = []
test_y = []

for i in range(len(import_data)):
    temp = []
    temp.append(import_data[i][1])
    y_data.append(temp)

raw_y = y_data
    
y_data = ohe.fit_transform(y_data)
y_data = y_data.toarray();

seq_length = 2
data_dim = 1
hidden_dim = 10
output_dim = 5
learning_rate = 0.001
iterations = 1000

dataX = []
dataY = []

for i in range(len(x_data) - seq_length):
    _x = x_data[i:i + seq_length]
    _y = y_data[i+seq_length]
    dataX.append(_x)
    dataY.append(_y)

train_size = int(len(dataY) * 0.7)
test_size = len(dataY) - train_size
trainX, testX = np.array(dataX[0:train_size]), np.array(dataX[train_size:len(dataX)])
trainY, testY = np.array(dataY[0:train_size]), np.array(dataY[train_size:len(dataY)])

print(np.shape(trainX))

X = tf.placeholder(tf.float32, [None, seq_length, data_dim])
Y = tf.placeholder(tf.float32, [None, output_dim])

W = tf.Variable(tf.random_normal([hidden_dim, output_dim]))
b = tf.Variable(tf.random_normal([output_dim]))

cell = tf.nn.rnn_cell.BasicLSTMCell(hidden_dim)
outputs, states = tf.nn.dynamic_rnn(cell, X, dtype=tf.float32)

outputs = tf.transpose(outputs, [1, 0, 2])
outputs = outputs[-1]
model = tf.matmul(outputs, W) + b

cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=model, labels=Y))
optimizer = tf.train.AdamOptimizer(learning_rate).minimize(cost)

is_correct = tf.equal(tf.argmax(model, 1), tf.argmax(Y, 1))
accuracy = tf.reduce_mean(tf.cast(is_correct, tf.float32))

tf.summary.scalar("accuracy", accuracy)

sess = tf.Session()
sess.run(tf.global_variables_initializer())

writer = tf.summary.FileWriter("./logs/swr_ratio_lstm_logs", sess.graph)
merged = tf.summary.merge_all()

for i in range(iterations):
    _, step_cost = sess.run([optimizer, cost], feed_dict={X:trainX, Y:trainY})
    print("[step: {}] cost: {}".format(i, step_cost))
    summary, acc = sess.run([merged, accuracy], feed_dict={X:testX, Y:testY})
    writer.add_summary(summary, i)
    if(i % 10 == 0):
        print('학습 정확도:', sess.run(accuracy, feed_dict={X:trainX, Y:trainY}))
        print('테스트 정확도:', sess.run(accuracy, feed_dict={X:testX, Y:testY}))

# cell = tf.contrib.rnn.BasicLSTMCell(num_units=hidden_dim, state_is_tuple=True, activation=tf.tanh)
# outputs, _states = tf.nn.dynamic_rnn(cell, X, dtype=tf.float32)
# Y_pred = tf.contrib.layers.fully_connected(outputs[:, -1], output_dim, activation_fn=tf.nn.softmax)

# loss = tf.reduce_sum(tf.square(Y_pred - Y))
# optimizer = tf.train.AdamOptimizer(learning_rate)
# train = optimizer.minimize(loss)

# targets = tf.placeholder(tf.float32, [None, 5])
# predictions = tf.placeholder(tf.float32, [None, 5])
# rmse = tf.sqrt(tf.reduce_mean(tf.square(targets - predictions)))

# with tf.Session() as sess:
#     init = tf.global_variables_initializer()
#     sess.run(init)
    
#     for i in range(iterations):
#         _, step_loss = sess.run([train, loss], feed_dict={X:trainX, Y:trainY})
#         print("[step: {}] loss: {}".format(i, step_loss))
        
#     test_predict = sess.run(Y_pred, feed_dict={X:testX})
#     rmse_val = sess.run(rmse, feed_dict={targets:testY, predictions:test_predict})
#     print("RMSE:{}".format(rmse_val))
    
#     print(sess.run(tf.argmax(test_predict, 1)))
#     print(len(dataX), len(trainX), len(testX))

/usr/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/usr/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/usr/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/usr/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/usr/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/usr/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected

(340, 2, 1)
[step: 0] cost: 2.3537700176239014
학습 정확도: 0.17058824
테스트 정확도: 0.05479452
[step: 1] cost: 2.3387763500213623
[step: 2] cost: 2.3240318298339844
[step: 3] cost: 2.309537172317505
[step: 4] cost: 2.295293092727661
[step: 5] cost: 2.281299114227295
[step: 6] cost: 2.267554998397827
[step: 7] cost: 2.2540595531463623
[step: 8] cost: 2.2408111095428467
[step: 9] cost: 2.2278079986572266
[step: 10] cost: 2.215047597885132
학습 정확도: 0.17058824
테스트 정확도: 0.05479452
[step: 11] cost: 2.2025279998779297
[step: 12] cost: 2.1902449131011963
[step: 13] cost: 2.178196907043457
[step: 14] cost: 2.16637921333313
[step: 15] cost: 2.1547892093658447
[step: 16] cost: 2.143423557281494
[step: 17] cost: 2.1322786808013916
[step: 18] cost: 2.121350049972534
[step: 19] cost: 2.110635995864868
[step: 20] cost: 2.100132942199707
학습 정확도: 0.17058824
테스트 정확도: 0.05479452
[step: 21] cost: 2.0898377895355225
[step: 22] cost: 2.0797486305236816
[step: 23] cost: 2.069861888885498
[step: 24] cost: 2.06017684936

[step: 244] cost: 1.5140037536621094
[step: 245] cost: 1.5139079093933105
[step: 246] cost: 1.513812780380249
[step: 247] cost: 1.5137178897857666
[step: 248] cost: 1.5136229991912842
[step: 249] cost: 1.51352858543396
[step: 250] cost: 1.5134345293045044
학습 정확도: 0.3529412
테스트 정확도: 0.41095892
[step: 251] cost: 1.5133410692214966
[step: 252] cost: 1.5132474899291992
[step: 253] cost: 1.5131545066833496
[step: 254] cost: 1.513061761856079
[step: 255] cost: 1.5129692554473877
[step: 256] cost: 1.5128769874572754
[step: 257] cost: 1.5127848386764526
[step: 258] cost: 1.512693166732788
[step: 259] cost: 1.5126017332077026
[step: 260] cost: 1.5125107765197754
학습 정확도: 0.3529412
테스트 정확도: 0.41095892
[step: 261] cost: 1.5124197006225586
[step: 262] cost: 1.5123291015625
[step: 263] cost: 1.512238621711731
[step: 264] cost: 1.512148380279541
[step: 265] cost: 1.5120586156845093
[step: 266] cost: 1.511968970298767
[step: 267] cost: 1.5118794441223145
[step: 268] cost: 1.51179039478302
[step: 269] 

[step: 508] cost: 1.4939982891082764
[step: 509] cost: 1.493928074836731
[step: 510] cost: 1.493857741355896
학습 정확도: 0.35588235
테스트 정확도: 0.41095892
[step: 511] cost: 1.4937872886657715
[step: 512] cost: 1.493716835975647
[step: 513] cost: 1.4936463832855225
[step: 514] cost: 1.4935758113861084
[step: 515] cost: 1.4935050010681152
[step: 516] cost: 1.4934345483779907
[step: 517] cost: 1.4933637380599976
[step: 518] cost: 1.4932928085327148
[step: 519] cost: 1.4932219982147217
[step: 520] cost: 1.493151068687439
학습 정확도: 0.35588235
테스트 정확도: 0.41095892
[step: 521] cost: 1.4930799007415771
[step: 522] cost: 1.4930089712142944
[step: 523] cost: 1.4929378032684326
[step: 524] cost: 1.4928667545318604
[step: 525] cost: 1.4927953481674194
[step: 526] cost: 1.4927239418029785
[step: 527] cost: 1.4926525354385376
[step: 528] cost: 1.4925810098648071
[step: 529] cost: 1.4925094842910767
[step: 530] cost: 1.4924376010894775
학습 정확도: 0.35588235
테스트 정확도: 0.41095892
[step: 531] cost: 1.4923659563064575

[step: 772] cost: 1.4607056379318237
[step: 773] cost: 1.460584044456482
[step: 774] cost: 1.4604599475860596
[step: 775] cost: 1.4603344202041626
[step: 776] cost: 1.4602097272872925
[step: 777] cost: 1.4600865840911865
[step: 778] cost: 1.459964394569397
[step: 779] cost: 1.459842562675476
[step: 780] cost: 1.4597201347351074
학습 정확도: 0.35
테스트 정확도: 0.39726028
[step: 781] cost: 1.4595974683761597
[step: 782] cost: 1.4594743251800537
[step: 783] cost: 1.4593517780303955
[step: 784] cost: 1.4592303037643433
[step: 785] cost: 1.4591091871261597
[step: 786] cost: 1.4589877128601074
[step: 787] cost: 1.4588656425476074
[step: 788] cost: 1.4587429761886597
[step: 789] cost: 1.4586201906204224
[step: 790] cost: 1.4584977626800537
학습 정확도: 0.34705883
테스트 정확도: 0.39726028
[step: 791] cost: 1.4583752155303955
[step: 792] cost: 1.4582525491714478
[step: 793] cost: 1.4581295251846313
[step: 794] cost: 1.4580059051513672
[step: 795] cost: 1.457882285118103
[step: 796] cost: 1.4577586650848389
[step: 

In [2]:
seq_length = 5
RNN
55.17

LSTM
55.17

seq_length = 4
RNN
704
52.74

LSTM
1000
54.11

seq_length = 3
RNN
52.05

LSTM
52.05

seq_length = 2
RNN
50.68

LSTM
48.63

NameError: name 'RNN' is not defined